In [ ]:
import numpy as np
import healpy as hp
from pathlib import Path

In [ ]:
datadir=Path("data/")
output_dir = Path("production-data/synch")
output_dir_raw = output_dir / "raw"

In [ ]:
output_nside = 2048

In [ ]:
output_lmax = int(min(2.5 * output_nside, 8192 * 2))

## Large scales

In [ ]:
alm_large_scale = hp.read_alm(
    output_dir_raw / f"synch_largescale_beta_alm_nside512_lmax1535_complex64.fits.gz",
    hdu=1,
)

In [ ]:
map_large_scale = hp.alm2map(alm_large_scale.astype(np.complex128), nside=output_nside)

## Small scales modulation

In [ ]:
modulate_alm = hp.read_alm(
    output_dir_raw / f"synch_temperature_modulation_alms_lmax768.fits.gz"
).astype(np.complex128)

## Small scales

In [ ]:
cl_small_scale = hp.read_cl(
    output_dir_raw / f"synch_small_scales_beta_cl_lmax16384.fits.gz"
)

In [ ]:
synalm_lmax = 8192 * 2  # for reproducibility
# synalm_lmax = output_lmax
seed = 444
np.random.seed(seed)

alm_small_scale = hp.synalm(
    cl_small_scale,
    lmax=synalm_lmax,
    new=True,
)

alm_small_scale = hp.almxfl(alm_small_scale, np.ones(output_lmax+1))
map_small_scale = hp.alm2map(alm_small_scale, nside=output_nside)
map_small_scale *= hp.alm2map(modulate_alm, output_nside)
assert np.isnan(map_small_scale).sum() == 0

## Combine scales

* Combine small and large scale maps
* Write output map

In [ ]:
output_map = map_large_scale + map_small_scale - 3.1

In [ ]:
hp.write_map(output_dir / f"synch_beta_nside{output_nside}.fits", output_map, dtype=np.float32, overwrite=True)

In [ ]:
from pysm3.utils import add_metadata

In [ ]:
add_metadata([output_dir / f"synch_beta_nside{output_nside}.fits"], coord="G", unit="", ref_freq="23 GHz")

In [ ]:
hp.mollview(map_large_scale, title="Large scale")
hp.mollview(map_small_scale, title="Small scale")
hp.mollview(output_map, title="Total")